In [1]:
import pandas as pd

# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

# Ler o arquivo CSV com as colunas especificadas, definindo a coluna 'classe' como object
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

In [4]:

X = vinhos.iloc[:, 1:]
y = vinhos['classe']


print("Dimensão dos dados:", vinhos.shape)
print(vinhos.head())

Dimensão dos dados: (178, 14)
  classe  alcool  acido_malico  cinzas  alcalinidade_de_cinzas  magnesio  \
0      1   14.23          1.71    2.43                    15.6       127   
1      1   13.20          1.78    2.14                    11.2       100   
2      1   13.16          2.36    2.67                    18.6       101   
3      1   14.37          1.95    2.50                    16.8       113   
4      1   13.24          2.59    2.87                    21.0       118   

   fenois_totais  flavanoides  fenois_nao_flavanoides  proantocianinas  \
0           2.80         3.06                    0.28             2.29   
1           2.65         2.76                    0.26             1.28   
2           2.80         3.24                    0.30             2.81   
3           3.85         3.49                    0.24             2.18   
4           2.80         2.69                    0.39             1.82   

   intensidade_de_cor  matiz  od280_od315_de_vinhos_diluidos  prolin

In [5]:

from sklearn.model_selection import KFold


random_states = [42, 17, 24]
kf_config = {rs: KFold(n_splits=10, shuffle=True, random_state=rs) for rs in random_states}


for random_state, kf in kf_config.items():
    print(f"\nRandom State: {random_state}")
    for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
        print(f"  Fold {fold}: {len(train_index)} treino, {len(test_index)} teste")



Random State: 42
  Fold 1: 160 treino, 18 teste
  Fold 2: 160 treino, 18 teste
  Fold 3: 160 treino, 18 teste
  Fold 4: 160 treino, 18 teste
  Fold 5: 160 treino, 18 teste
  Fold 6: 160 treino, 18 teste
  Fold 7: 160 treino, 18 teste
  Fold 8: 160 treino, 18 teste
  Fold 9: 161 treino, 17 teste
  Fold 10: 161 treino, 17 teste

Random State: 17
  Fold 1: 160 treino, 18 teste
  Fold 2: 160 treino, 18 teste
  Fold 3: 160 treino, 18 teste
  Fold 4: 160 treino, 18 teste
  Fold 5: 160 treino, 18 teste
  Fold 6: 160 treino, 18 teste
  Fold 7: 160 treino, 18 teste
  Fold 8: 160 treino, 18 teste
  Fold 9: 161 treino, 17 teste
  Fold 10: 161 treino, 17 teste

Random State: 24
  Fold 1: 160 treino, 18 teste
  Fold 2: 160 treino, 18 teste
  Fold 3: 160 treino, 18 teste
  Fold 4: 160 treino, 18 teste
  Fold 5: 160 treino, 18 teste
  Fold 6: 160 treino, 18 teste
  Fold 7: 160 treino, 18 teste
  Fold 8: 160 treino, 18 teste
  Fold 9: 161 treino, 17 teste
  Fold 10: 161 treino, 17 teste


In [6]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


ks = [3, 5]


resultados = []

for random_state, kf in kf_config.items():
    print(f"\nRandom State: {random_state}")

    for k in ks:
        acuracias = []

        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]


            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)


            y_pred = knn.predict(X_test)
            acuracia = accuracy_score(y_test, y_pred)
            acuracias.append(acuracia)


        media = sum(acuracias) / len(acuracias)
        desvio_padrao = pd.Series(acuracias).std()
        resultados.append({
            'random_state': random_state,
            'k': k,
            'media_acuracia': media,
            'desvio_padrao': desvio_padrao
        })

        print(f"  k={k} -> Média da Acurácia: {media:.4f}, Desvio Padrão: {desvio_padrao:.4f}")



Random State: 42
  k=3 -> Média da Acurácia: 0.7036, Desvio Padrão: 0.1467
  k=5 -> Média da Acurácia: 0.6641, Desvio Padrão: 0.1055

Random State: 17
  k=3 -> Média da Acurácia: 0.6987, Desvio Padrão: 0.1097
  k=5 -> Média da Acurácia: 0.6876, Desvio Padrão: 0.1182

Random State: 24
  k=3 -> Média da Acurácia: 0.7127, Desvio Padrão: 0.1021
  k=5 -> Média da Acurácia: 0.7127, Desvio Padrão: 0.0991


In [7]:

import pandas as pd


df_resultados = pd.DataFrame(resultados)


for random_state in random_states:
    print(f"\nResultados para Random State {random_state}:")
    df_rs = df_resultados[df_resultados['random_state'] == random_state]
    print(df_rs)

    melhor_k = df_rs.loc[df_rs['media_acuracia'].idxmax()]
    print(f"Melhor configuração: k={melhor_k['k']}, Média da Acurácia: {melhor_k['media_acuracia']:.4f}")


melhor_resultado = df_resultados.loc[df_resultados['media_acuracia'].idxmax()]
print("\nMelhor configuração geral:")
print(melhor_resultado)



Resultados para Random State 42:
   random_state  k  media_acuracia  desvio_padrao
0            42  3        0.703595       0.146716
1            42  5        0.664052       0.105488
Melhor configuração: k=3.0, Média da Acurácia: 0.7036

Resultados para Random State 17:
   random_state  k  media_acuracia  desvio_padrao
2            17  3        0.698693       0.109658
3            17  5        0.687582       0.118182
Melhor configuração: k=3.0, Média da Acurácia: 0.6987

Resultados para Random State 24:
   random_state  k  media_acuracia  desvio_padrao
4            24  3        0.712745       0.102053
5            24  5        0.712745       0.099056
Melhor configuração: k=3.0, Média da Acurácia: 0.7127

Melhor configuração geral:
random_state      24.000000
k                  3.000000
media_acuracia     0.712745
desvio_padrao      0.102053
Name: 4, dtype: float64
